In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #for viualisasi
import matplotlib.pyplot as plt #for viualisasi
%matplotlib inline #for viualisasi
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import and show the data

In [ ]:
#import data
data = pd.read_csv('/kaggle/input/antam-stock-market-by-kitto/ANTM.JK.csv')

In [ ]:
#show data
data

# Cleansing Data

> if we want to explore data, first we must treat Nan that can affect to data

In [ ]:
#check Nan in columns 
data.info()
#you also can use data.isna() for check nan

from info we know that there is 1 Nan in Columns Open until Volume.
Still confuse how to know Nan from data info ?? okay, lets see rangeIndex , there is 3808 entries, so all of the columns must have Non-Null Count 3808. And in columns open until Volume just have 3807 Non-Null, its mean they have 1 Nan.
from info we also know about Data type that can use for visualisation

In [ ]:
#show data that have Nan Value
data[data.isna().any(axis=1)]

In [ ]:
#delete Nan, I decided to delete Nan because they have Nan in 1 row, so the data cant we use to explore
data.dropna(inplace= True)

In [ ]:
#check again about nan
data.isna().sum()

YAY!! there is not Nan Values

> Hmmm Data Type from columns date is object, lets convert it to Datetime

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data.info()

In [ ]:
#you can use describe to see minimum, maximum values in all of the columns. i just use describe because the all of Data Type columns is float except column date
data.describe()

> lets see about the correlation columns, sometimes new trader study about technical from stock, so i think imortant to know the correlation from all of the columns

In [ ]:
sns.heatmap(data[['Open','High','Low','Close','Adj Close','Volume']].corr(),annot=True)

from heatmap we can know that all of the Columns except column Volume has strong correlation with other.

In [ ]:
#check the correlation with scatterplot
sns.pairplot(data[data['Open']==data['Open']])

In [ ]:
#lets check when highest price occurs
data[data['High']==data['High'].max()]

In [ ]:
#lets check when lowest price occurs
data[data['Low']==data['Low'].min()]

In [ ]:
#lets visual the high price
sns.lineplot(data = data, x = 'Date', y = 'High')

In [ ]:
#lets visual the Close price
sns.lineplot(data = data, x = 'Date', y = 'Low')

what month that good to buy and taking profit ANTM Stock ?

In [ ]:
x = data[['Close','Date']].groupby(data['Date'].dt.month).mean()
x


In [ ]:
temp = data.groupby(data['Date'].dt.month).mean().reset_index()

plt.figure(figsize = (8,8))
sns.lineplot(data= temp, x = temp['Date'], y = temp['Close'])
plt.title('Price base on Month')
plt.show()

From the lineplot, we could find that highest close price in 4th month (April) and lowest close price in 9th month(September). Thus we could use this reference for near future to take profit at April and buy at September.


# UJI T-TES

> Lets check, is the open price and close price has same value? 

Hypothesis:
* H0 : open price affect close price (U0=U1)
* H1 : Open price not affect close price (U0 != U1)

In [ ]:
from scipy.stats import kstest

In [ ]:
normal_check =[]
for i in data[['Open','Close']].columns:
    key, val = kstest(data[i], 'norm')
    if val < 0.05: 
        normal_check.append('Norm Distribution')
    else :
        normal_check.append('Not Norm Distribution')

In [ ]:
normal_check

The data is not norm distribution, so we use Mannwhitneyu for T-Tes

In [ ]:
from scipy.stats import mannwhitneyu
a,pval = mannwhitneyu(data['Open'],data['Close'])

In [ ]:
pval>0.05 

pval < 0.05 so reject Null Hypothesis
and the conclusion is accept Alternatif Hypothesis (Open price not affect Close Price)

> is the corona affect antm price?? we can know it with T-Test.
First we check Close Price ANTM 3months ago.

In [ ]:
a = data[data['Date'].dt.year==2017]['Close'].mean()
a

In [ ]:
b = data[data['Date'].dt.year==2018]['Close'].mean()
b

In [ ]:
c = data[data['Date'].dt.year==2019]['Close'].mean()
c

In [ ]:
new =pd.DataFrame ([a,b,c] ,index= ['2017','2018','2019'],columns= ['Close Mean'])
new

New DataFrame show that ANTM always increased every year, so normally in 2020 ANTM should increased
but we will check it and assume that Corona affect ANTM price

Hypothesis:
* h0 : close price 2020 >= 2019
* h1 : close price 2020 < 2019

we will use mannwhitneyu method because we will compare 2 variable that have non parametic values

In [ ]:
a,pval = mannwhitneyu(data[data['Date'].dt.year==2020]['Close'],data[data['Date'].dt.year==2019]['Close'], alternative = 'less')


In [ ]:
pval<0.05

True, its mean we reject Null hypothesis( Close price in 2020 is lower than Close price in 2019)
we can concluse that corona affect to ANTM stock, cause there is lowering.

> Lets check, about volume
is corona also affect volume in ANTM Stock ? we can check from delta Volume ANTM in 2020 with 2019

In [ ]:
(data[data['Date'].dt.year==2020]['Volume'].mean())-(data[data['Date'].dt.year==2019]['Volume'].mean())


WAW!! its positif! its mean many people still trust and Invest in ANTM Stock, although its pandemic.